In [1]:
import sys,os
if 'google.colab' in sys.modules:
  from google.colab import drive
  drive.mount('/content/gdrive')
  path_to_file = '/content/gdrive/My Drive/AI Sem II/NLP/A2'
  print(path_to_file)
  os.chdir(path_to_file)
  !pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/AI Sem II/NLP/A2
/content/gdrive/My Drive/AI Sem II/NLP/A2


In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
data_ru = 'data/training/news-commentary-v9.ru-en.ru'
data_en = 'data/training/news-commentary-v9.ru-en.en'

with open(data_ru, 'rb') as ru: 
  sents_ru = [line.decode("utf-8") for line in ru]      
 # sents_cs = [value for value in sents_cs if value != '']   
with open(data_en, 'rb') as en: 
  sents_en = [line.decode("utf-8") for line in en]
 # sents_en = [value for value in sents_en if value != '']
len(sents_en), len(sents_ru)

(165602, 165602)

In [4]:
#max length of string 
length_ru = [len(i.split()) for i in sents_ru]
max(length_ru)

160

In [5]:
length_en = [len(i.split()) for i in sents_en]
max(length_en)

171

In [6]:
SOS_token = 0
EOS_token = 1

class Lang:
  def __init__(self, name):
    self.name = name
    self.word2index = {}
    self.word2count = {}
    self.index2word = {0: "SOS", 1: "EOS"}
    self.n_words = 2

  def addSentence(self, sentence):
    
    for word in sentence.split(' '):
      self.addWord(word)
  
  def addWord(self, word):
    if word not in self.word2index:
      self.word2index[word] = self.n_words
      self.word2count[word] = 1
      self.index2word[self.n_words] = word 
      self.n_words += 1
    else:
      self.word2count[word] += 1

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [7]:
def sent_pairs(lang1=sents_ru, lang2=sents_en):
  pairs = []
  for i, (cs_sent, en_sent) in enumerate(zip(lang1, lang2)):
    #if i < 100:
      #for russia
      en_sent = unicodeToAscii(en_sent.lower().strip()) #for russian
      en_sent = re.sub(r"([.!?])", r" \1", en_sent)#for russian
      en_sent = re.sub(r"[^a-zA-Z.!?]+", r" ", en_sent)#for russian
      cs_sent = re.sub(r"([.!?])", r" \1", cs_sent)#for russian
      pairs.append([cs_sent, en_sent])

  # for others
  # pairs = [[normalizeString(s) for s in line] for line in pairs] # for others
  input_lang1 = Lang('ru')
  output_lang1 = Lang('en')

  input_lang2 = Lang('ru')
  output_lang2 = Lang('en')

  input_lang3 = Lang('ru')
  output_lang3 = Lang('en')

  input_lang4 = Lang('ru')
  output_lang4 = Lang('en')

  input_lang5 = Lang('ru')
  output_lang5 = Lang('en')

     
  return input_lang1, output_lang1, input_lang2, output_lang2, input_lang3, output_lang3, input_lang4, output_lang4, input_lang5, output_lang5, pairs
 

The full process for preparing the data is:

-  Read text file and split into lines, split lines into pairs
-  Normalize text, filter by length and content
-  Make word lists from sentences in pairs

In [8]:
 def prepareData(lang1=sents_ru, lang2=sents_en):
    input_lang1, output_lang1, input_lang2, output_lang2, input_lang3, output_lang3, input_lang4, output_lang4, input_lang5, output_lang5, pairs = sent_pairs(lang1, lang2)
    print("Read %s sentence pairs" % len(pairs))

    #Assumption: as the dataset is news commentary on different topics, there is highly unlikely exactly the same sentences 
    # collect test pairs
    num_test = int(len(pairs)*0.2)
    print("Number of test pairs:", num_test)
    random.seed(4)
    random.shuffle(pairs)
    
    #fold 1
    test_pairs1 = pairs[:num_test]
     # collect train pairs
    train_pairs1 = pairs[num_test:]
    print("Number of train pairs:", len(train_pairs1))
    print("Counting words...")

    for pair in train_pairs1:      
      input_lang1.addSentence(pair[0])
      output_lang1.addSentence(pair[1])
    print("Counted words:")
    print(input_lang1.name, input_lang1.n_words)
    print(output_lang1.name, output_lang1.n_words)

    #fold 2
    test_pairs2 = pairs[num_test:num_test*2]
     # collect train pairs
    train_pairs2 = pairs[:num_test]
    for x in pairs[num_test*2:]:
      train_pairs2.append(x)
    print("Number of train pairs:", len(train_pairs2))
    print("Counting words...")


    for pair in train_pairs2:      
      input_lang2.addSentence(pair[0])
      output_lang2.addSentence(pair[1])
    print("Counted words:")
    print(input_lang2.name, input_lang2.n_words)
    print(output_lang2.name, output_lang2.n_words)

    #fold 3
    test_pairs3 = pairs[num_test*2:num_test*3]
     # collect train pairs
    train_pairs3 = pairs[:num_test*2]
    for x in pairs[num_test*3:]:
      train_pairs3.append(x)
    print("Number of train pairs:", len(train_pairs3))
    print("Counting words...")


    for pair in train_pairs3:      
      input_lang3.addSentence(pair[0])
      output_lang3.addSentence(pair[1])
    print("Counted words:")
    print(input_lang3.name, input_lang3.n_words)
    print(output_lang3.name, output_lang3.n_words)


    #fold 4
    test_pairs4 = pairs[num_test*3:num_test*4]
     # collect train pairs
    train_pairs4 = pairs[:num_test*3]
    for x in pairs[num_test*4:]:
      train_pairs4.append(x)
    print("Number of train pairs:", len(train_pairs4))
    print("Counting words...")


    for pair in train_pairs4:      
      input_lang4.addSentence(pair[0])
      output_lang4.addSentence(pair[1])
    print("Counted words:")
    print(input_lang4.name, input_lang4.n_words)
    print(output_lang4.name, output_lang4.n_words)

    #fold 5
    test_pairs5 = pairs[num_test*4:]
     # collect train pairs
    train_pairs5 = pairs[:num_test*4]
    print("Number of train pairs:", len(train_pairs5))
    print("Counting words...")


    for pair in train_pairs5:      
      input_lang5.addSentence(pair[0])
      output_lang5.addSentence(pair[1])
    print("Counted words:")
    print(input_lang5.name, input_lang5.n_words)
    print(output_lang5.name, output_lang5.n_words)


    return (input_lang1, output_lang1, input_lang2, output_lang2, input_lang3, output_lang3, input_lang4, output_lang4,input_lang5, output_lang5,train_pairs1, 
            test_pairs1, train_pairs2, test_pairs2, train_pairs3, test_pairs3,train_pairs4, test_pairs4,train_pairs5, test_pairs5)


(input_lang1, output_lang1, input_lang2, output_lang2, input_lang3, output_lang3, input_lang4, output_lang4, input_lang5, output_lang5, train_pairs1, test_pairs1, train_pairs2,
 test_pairs2,train_pairs3, test_pairs3, train_pairs4, test_pairs4,train_pairs5, test_pairs5) = prepareData(sents_ru, sents_en)
print(random.choice(train_pairs1))
print(random.choice(train_pairs2))
print(random.choice(train_pairs3))
print(random.choice(train_pairs4))
print(random.choice(train_pairs5))

Read 165602 sentence pairs
Number of test pairs: 33120
Number of train pairs: 132482
Counting words...
Counted words:
ru 211664
en 42098
Number of train pairs: 132482
Counting words...
Counted words:
ru 211530
en 42042
Number of train pairs: 132482
Counting words...
Counted words:
ru 211567
en 42086
Number of train pairs: 132482
Counting words...
Counted words:
ru 211768
en 42154
Number of train pairs: 132480
Counting words...
Counted words:
ru 211830
en 42243
['Скорее, Путина заботит баланс второй мировой войны и сталинизма в советской истории .\n', 'rather what concerns putin is the balancing of wwii and stalinism in soviet history .']
['Кроме того, вопросы повышения качества образования и квалификации, особенно в отношении девочек и женщин, остаются одними из приоритетных .\n', 'moreover the challenge of raising educational quality and attainment especially of girls and women remains a priority .']
['На протяжении слишком многих лет Балтийское море было тупиком на политической карте

In [9]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
MAX_LENGTH = 171
# Multiplicative attention https://blog.floydhub.com/attention-mechanism/
class AttnDecoderRNN4(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN4, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        #################
        self.fc = nn.Linear(hidden_size, hidden_size, bias=False) #  Multiplicative
        ##################
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        
        x=self.fc(hidden)
        #print(x.view(1,-1,1).shape,encoder_outputs.unsqueeze(0).shape )
        attn_weights = F.softmax(
            #################################
             encoder_outputs.unsqueeze(0).bmm(x.view(1,-1,1)).view(1,-1), dim=1)
            #################################
       # print(attn_weights.view(1,-1,1).shape,encoder_outputs.unsqueeze(0).shape)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
        
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ') if word in lang.word2index]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang1, pair[0])
    target_tensor = tensorFromSentence(output_lang1, pair[1])
    return (input_tensor, target_tensor)

teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_pairs1))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [10]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang1, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang1.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self, alpha=1.0):
        reward = 0
        # Add here a function for shaping a reward

        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward
    
    def __lt__(self, other):
        return self.eval() < other.eval()
    
    
from queue import PriorityQueue

def evaluate_beam_search(encoder, decoder, sentence, max_length=MAX_LENGTH, beam_size=2):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang1, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        # Number of sentence to generate
        endnodes = []
        number_required = 1
        
        # starting node -  hidden vector, previous node, word id, logp, length
        node = BeamSearchNode(decoder_hidden, None, decoder_input, 0, 1)
        nodes = PriorityQueue()

        # start the queue
        nodes.put((-node.eval(), node))
        qsize = 1
        
        # start beam search
        while True:
            # give up when decoding takes too long
            if qsize > 2000: break

            # fetch the best node
            score, n = nodes.get()
            decoder_input = n.wordid
            decoder_hidden = n.h

            if n.wordid.item() == EOS_token and n.prevNode != None:
                endnodes.append((score, n))
                # if we reached maximum # of sentences required
                if len(endnodes) >= number_required:
                    break
                else:
                    continue
            #elif n.leng > max_length:
            #    continue

            # decode for one step using decoder
            decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_outputs)

            # PUT HERE REAL BEAM SEARCH OF TOP
            log_prob, indexes = torch.topk(decoder_output, beam_size)
            nextnodes = []

            for new_k in range(beam_size):
                decoded_t = indexes[0][new_k].view(1, -1)
                log_p = log_prob[0][new_k].item()

                node = BeamSearchNode(decoder_hidden, n, decoded_t, n.logp + log_p, n.leng + 1)
                score = -node.eval()
                nextnodes.append((score, node))

            # put them into queue
            for i in range(len(nextnodes)):
                score, nn = nextnodes[i]
                nodes.put((score, nn))
                
            # increase qsize
            qsize += len(nextnodes) - 1
            
        # choose nbest paths, back trace them
        if len(endnodes) == 0:
            endnodes = [nodes.get() for _ in range(number_required)]

        _, n = endnodes[0]
        utterance = []
        utterance.append(output_lang1.index2word[n.wordid.item()])
        
        # back trace
        while n.prevNode != None:
            n = n.prevNode
            utterance.append(output_lang1.index2word[n.wordid.item()])

        utterance = utterance[::-1]
            
    return utterance, None

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(test_pairs1)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

from nltk.translate.bleu_score import corpus_bleu

def evaluateBleu(encoder, decoder):
    references, candidates = [], []
    i= 0
    for sent_eng, sents_fre in test_pairs1:
        i=i+1
        sents_fre = [sent_fre.split(' ') for sent_fre in [sents_fre]]
        output_words, _ = evaluate(encoder, decoder, sent_eng)
        references.append(sents_fre)
        candidates.append(output_words)
        if i%1000==0:
          print(i)
    #return bleu 1, bleu 2, bleu 3
    score1 = corpus_bleu(references, candidates, weights=(1.0, 0, 0))
    score2 = corpus_bleu(references, candidates, weights=(0.5, 0.5, 0))
    score3 = corpus_bleu(references, candidates, weights=(0.33, 0.33, 0.33))
    #print (references,  candidates)
    scores = [score1, score2, score3]
    return scores

from nltk.translate.bleu_score import corpus_bleu

def evaluateBleu_beam_search(encoder, decoder, beam_size):
    references, candidates = [], []
    for sent_eng, sents_fre in test_pairs1:
        sents_fre = [sent_fre.split(' ') for sent_fre in [sents_fre]]
        output_words, _ = evaluate_beam_search(encoder, decoder, sent_eng, beam_size=beam_size)
        references.append(sents_fre)
        candidates.append(output_words)
    #return bleu 1, bleu 2, bleu 3
    score1 = corpus_bleu(references, candidates, weights=(1.0, 0, 0))
    score2 = corpus_bleu(references, candidates, weights=(0.5, 0.5, 0))
    score3 = corpus_bleu(references, candidates, weights=(0.33, 0.33, 0.33))
    #print (references,  candidates)
    scores = [score1, score2, score3]
    return scores

In [11]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang1.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN4(hidden_size, output_lang1.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=1000) #reduce number of epoch
print("Fold 1 Bleu-1, Bleu-2, Bleu-3 scores are ",evaluateBleu(encoder1, attn_decoder1))

1m 26s (- 106m 20s) (1000 1%) 6.1784
2m 46s (- 101m 3s) (2000 2%) 5.4603
4m 4s (- 97m 53s) (3000 4%) 5.3629
5m 23s (- 95m 36s) (4000 5%) 5.3781
6m 46s (- 94m 47s) (5000 6%) 5.4054
8m 8s (- 93m 35s) (6000 8%) 5.2756
9m 26s (- 91m 42s) (7000 9%) 5.3038
10m 48s (- 90m 30s) (8000 10%) 5.4157
12m 10s (- 89m 14s) (9000 12%) 5.2913
13m 33s (- 88m 5s) (10000 13%) 5.2577
14m 55s (- 86m 49s) (11000 14%) 5.3700
16m 18s (- 85m 36s) (12000 16%) 5.3190
17m 41s (- 84m 22s) (13000 17%) 5.3308
19m 5s (- 83m 12s) (14000 18%) 5.4564
20m 28s (- 81m 54s) (15000 20%) 5.3916
21m 52s (- 80m 40s) (16000 21%) 5.3770
23m 19s (- 79m 35s) (17000 22%) 5.5140
24m 43s (- 78m 19s) (18000 24%) 5.4723
26m 9s (- 77m 6s) (19000 25%) 5.5833
27m 35s (- 75m 52s) (20000 26%) 5.4816
28m 59s (- 74m 31s) (21000 28%) 5.4941
30m 24s (- 73m 16s) (22000 29%) 5.5293
31m 51s (- 72m 2s) (23000 30%) 5.5135
33m 16s (- 70m 41s) (24000 32%) 5.4933
34m 42s (- 69m 24s) (25000 33%) 5.3580
36m 7s (- 68m 4s) (26000 34%) 5.3539
37m 32s (- 66m 43

###Fold2

In [12]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
MAX_LENGTH = 171
# Multiplicative attention https://blog.floydhub.com/attention-mechanism/
class AttnDecoderRNN4(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN4, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        #################
        self.fc = nn.Linear(hidden_size, hidden_size, bias=False) #  Multiplicative
        ##################
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        
        x=self.fc(hidden)
        #print(x.view(1,-1,1).shape,encoder_outputs.unsqueeze(0).shape )
        attn_weights = F.softmax(
            #################################
             encoder_outputs.unsqueeze(0).bmm(x.view(1,-1,1)).view(1,-1), dim=1)
            #################################
       # print(attn_weights.view(1,-1,1).shape,encoder_outputs.unsqueeze(0).shape)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
        
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ') if word in lang.word2index]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang2, pair[0])
    target_tensor = tensorFromSentence(output_lang2, pair[1])
    return (input_tensor, target_tensor)

teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_pairs2))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [13]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang2, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang2.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self, alpha=1.0):
        reward = 0
        # Add here a function for shaping a reward

        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward
    
    def __lt__(self, other):
        return self.eval() < other.eval()
    
    
from queue import PriorityQueue

def evaluate_beam_search(encoder, decoder, sentence, max_length=MAX_LENGTH, beam_size=2):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang2, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        # Number of sentence to generate
        endnodes = []
        number_required = 1
        
        # starting node -  hidden vector, previous node, word id, logp, length
        node = BeamSearchNode(decoder_hidden, None, decoder_input, 0, 1)
        nodes = PriorityQueue()

        # start the queue
        nodes.put((-node.eval(), node))
        qsize = 1
        
        # start beam search
        while True:
            # give up when decoding takes too long
            if qsize > 2000: break

            # fetch the best node
            score, n = nodes.get()
            decoder_input = n.wordid
            decoder_hidden = n.h

            if n.wordid.item() == EOS_token and n.prevNode != None:
                endnodes.append((score, n))
                # if we reached maximum # of sentences required
                if len(endnodes) >= number_required:
                    break
                else:
                    continue
            #elif n.leng > max_length:
            #    continue

            # decode for one step using decoder
            decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_outputs)

            # PUT HERE REAL BEAM SEARCH OF TOP
            log_prob, indexes = torch.topk(decoder_output, beam_size)
            nextnodes = []

            for new_k in range(beam_size):
                decoded_t = indexes[0][new_k].view(1, -1)
                log_p = log_prob[0][new_k].item()

                node = BeamSearchNode(decoder_hidden, n, decoded_t, n.logp + log_p, n.leng + 1)
                score = -node.eval()
                nextnodes.append((score, node))

            # put them into queue
            for i in range(len(nextnodes)):
                score, nn = nextnodes[i]
                nodes.put((score, nn))
                
            # increase qsize
            qsize += len(nextnodes) - 1
            
        # choose nbest paths, back trace them
        if len(endnodes) == 0:
            endnodes = [nodes.get() for _ in range(number_required)]

        _, n = endnodes[0]
        utterance = []
        utterance.append(output_lang2.index2word[n.wordid.item()])
        
        # back trace
        while n.prevNode != None:
            n = n.prevNode
            utterance.append(output_lang2.index2word[n.wordid.item()])

        utterance = utterance[::-1]
            
    return utterance, None

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(test_pairs2)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

from nltk.translate.bleu_score import corpus_bleu

def evaluateBleu(encoder, decoder):
    references, candidates = [], []
    i= 0
    for sent_eng, sents_fre in test_pairs2:
        i=i+1
        sents_fre = [sent_fre.split(' ') for sent_fre in [sents_fre]]
        output_words, _ = evaluate(encoder, decoder, sent_eng)
        references.append(sents_fre)
        candidates.append(output_words)
        if i%1000==0:
          print(i)
    #return bleu 1, bleu 2, bleu 3
    score1 = corpus_bleu(references, candidates, weights=(1.0, 0, 0))
    score2 = corpus_bleu(references, candidates, weights=(0.5, 0.5, 0))
    score3 = corpus_bleu(references, candidates, weights=(0.33, 0.33, 0.33))
    #print (references,  candidates)
    scores = [score1, score2, score3]
    return scores

from nltk.translate.bleu_score import corpus_bleu

def evaluateBleu_beam_search(encoder, decoder, beam_size):
    references, candidates = [], []
    for sent_eng, sents_fre in test_pairs2:
        sents_fre = [sent_fre.split(' ') for sent_fre in [sents_fre]]
        output_words, _ = evaluate_beam_search(encoder, decoder, sent_eng, beam_size=beam_size)
        references.append(sents_fre)
        candidates.append(output_words)
    #return bleu 1, bleu 2, bleu 3
    score1 = corpus_bleu(references, candidates, weights=(1.0, 0, 0))
    score2 = corpus_bleu(references, candidates, weights=(0.5, 0.5, 0))
    score3 = corpus_bleu(references, candidates, weights=(0.33, 0.33, 0.33))
    #print (references,  candidates)
    scores = [score1, score2, score3]
    return scores

In [14]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang2.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN4(hidden_size, output_lang2.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=1000) #reduce number of epoch
print("Fold 2 Bleu-1, Bleu-2, Bleu-3 scores are ",evaluateBleu(encoder1, attn_decoder1))

1m 26s (- 106m 59s) (1000 1%) 6.0545
2m 46s (- 101m 8s) (2000 2%) 5.6105
4m 7s (- 98m 53s) (3000 4%) 5.4929
5m 27s (- 96m 49s) (4000 5%) 5.2880
6m 51s (- 96m 1s) (5000 6%) 5.4638
8m 12s (- 94m 27s) (6000 8%) 5.3466
9m 34s (- 93m 4s) (7000 9%) 5.4857
10m 57s (- 91m 49s) (8000 10%) 5.3727
12m 18s (- 90m 14s) (9000 12%) 5.4143
13m 39s (- 88m 45s) (10000 13%) 5.2929
15m 0s (- 87m 20s) (11000 14%) 5.3680
16m 24s (- 86m 10s) (12000 16%) 5.3656
17m 48s (- 84m 56s) (13000 17%) 5.3933
19m 12s (- 83m 41s) (14000 18%) 5.4408
20m 35s (- 82m 22s) (15000 20%) 5.5049
22m 0s (- 81m 9s) (16000 21%) 5.5810
23m 26s (- 79m 59s) (17000 22%) 5.5638
24m 50s (- 78m 38s) (18000 24%) 5.5080
26m 18s (- 77m 32s) (19000 25%) 5.5667
27m 45s (- 76m 21s) (20000 26%) 5.5024
29m 11s (- 75m 3s) (21000 28%) 5.5193
30m 36s (- 73m 43s) (22000 29%) 5.4705
32m 3s (- 72m 28s) (23000 30%) 5.5511
33m 31s (- 71m 14s) (24000 32%) 5.5970
34m 54s (- 69m 49s) (25000 33%) 5.3641
36m 19s (- 68m 27s) (26000 34%) 5.4867
37m 44s (- 67m 5

###Fold 3

In [15]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
MAX_LENGTH = 171
# Multiplicative attention https://blog.floydhub.com/attention-mechanism/
class AttnDecoderRNN4(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN4, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        #################
        self.fc = nn.Linear(hidden_size, hidden_size, bias=False) #  Multiplicative
        ##################
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        
        x=self.fc(hidden)
        #print(x.view(1,-1,1).shape,encoder_outputs.unsqueeze(0).shape )
        attn_weights = F.softmax(
            #################################
             encoder_outputs.unsqueeze(0).bmm(x.view(1,-1,1)).view(1,-1), dim=1)
            #################################
       # print(attn_weights.view(1,-1,1).shape,encoder_outputs.unsqueeze(0).shape)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
        
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
        
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ') if word in lang.word2index]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang3, pair[0])
    target_tensor = tensorFromSentence(output_lang3, pair[1])
    return (input_tensor, target_tensor)

teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_pairs3))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [16]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang3, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang3.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self, alpha=1.0):
        reward = 0
        # Add here a function for shaping a reward

        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward
    
    def __lt__(self, other):
        return self.eval() < other.eval()
    
    
from queue import PriorityQueue

def evaluate_beam_search(encoder, decoder, sentence, max_length=MAX_LENGTH, beam_size=2):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang3, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        # Number of sentence to generate
        endnodes = []
        number_required = 1
        
        # starting node -  hidden vector, previous node, word id, logp, length
        node = BeamSearchNode(decoder_hidden, None, decoder_input, 0, 1)
        nodes = PriorityQueue()

        # start the queue
        nodes.put((-node.eval(), node))
        qsize = 1
        
        # start beam search
        while True:
            # give up when decoding takes too long
            if qsize > 2000: break

            # fetch the best node
            score, n = nodes.get()
            decoder_input = n.wordid
            decoder_hidden = n.h

            if n.wordid.item() == EOS_token and n.prevNode != None:
                endnodes.append((score, n))
                # if we reached maximum # of sentences required
                if len(endnodes) >= number_required:
                    break
                else:
                    continue
            #elif n.leng > max_length:
            #    continue

            # decode for one step using decoder
            decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_outputs)

            # PUT HERE REAL BEAM SEARCH OF TOP
            log_prob, indexes = torch.topk(decoder_output, beam_size)
            nextnodes = []

            for new_k in range(beam_size):
                decoded_t = indexes[0][new_k].view(1, -1)
                log_p = log_prob[0][new_k].item()

                node = BeamSearchNode(decoder_hidden, n, decoded_t, n.logp + log_p, n.leng + 1)
                score = -node.eval()
                nextnodes.append((score, node))

            # put them into queue
            for i in range(len(nextnodes)):
                score, nn = nextnodes[i]
                nodes.put((score, nn))
                
            # increase qsize
            qsize += len(nextnodes) - 1
            
        # choose nbest paths, back trace them
        if len(endnodes) == 0:
            endnodes = [nodes.get() for _ in range(number_required)]

        _, n = endnodes[0]
        utterance = []
        utterance.append(output_lang3.index2word[n.wordid.item()])
        
        # back trace
        while n.prevNode != None:
            n = n.prevNode
            utterance.append(output_lang3.index2word[n.wordid.item()])

        utterance = utterance[::-1]
            
    return utterance, None

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(test_pairs3)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

from nltk.translate.bleu_score import corpus_bleu

def evaluateBleu(encoder, decoder):
    references, candidates = [], []
    i= 0
    for sent_eng, sents_fre in test_pairs3:
        i=i+1
        sents_fre = [sent_fre.split(' ') for sent_fre in [sents_fre]]
        output_words, _ = evaluate(encoder, decoder, sent_eng)
        references.append(sents_fre)
        candidates.append(output_words)
        if i%1000==0:
          print(i)
    #return bleu 1, bleu 2, bleu 3
    score1 = corpus_bleu(references, candidates, weights=(1.0, 0, 0))
    score2 = corpus_bleu(references, candidates, weights=(0.5, 0.5, 0))
    score3 = corpus_bleu(references, candidates, weights=(0.33, 0.33, 0.33))
    #print (references,  candidates)
    scores = [score1, score2, score3]
    return scores

from nltk.translate.bleu_score import corpus_bleu

def evaluateBleu_beam_search(encoder, decoder, beam_size):
    references, candidates = [], []
    for sent_eng, sents_fre in test_pairs3:
        sents_fre = [sent_fre.split(' ') for sent_fre in [sents_fre]]
        output_words, _ = evaluate_beam_search(encoder, decoder, sent_eng, beam_size=beam_size)
        references.append(sents_fre)
        candidates.append(output_words)
    #return bleu 1, bleu 2, bleu 3
    score1 = corpus_bleu(references, candidates, weights=(1.0, 0, 0))
    score2 = corpus_bleu(references, candidates, weights=(0.5, 0.5, 0))
    score3 = corpus_bleu(references, candidates, weights=(0.33, 0.33, 0.33))
    #print (references,  candidates)
    scores = [score1, score2, score3]
    return scores

In [17]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang3.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN4(hidden_size, output_lang3.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=1000) #reduce number of epoch
print("Fold 3 Bleu-1, Bleu-2, Bleu-3 scores are ",evaluateBleu(encoder1, attn_decoder1))

1m 25s (- 105m 53s) (1000 1%) 5.9143
2m 48s (- 102m 19s) (2000 2%) 5.5622
4m 8s (- 99m 24s) (3000 4%) 5.4702
5m 28s (- 97m 3s) (4000 5%) 5.2404
6m 50s (- 95m 43s) (5000 6%) 5.5198
8m 11s (- 94m 16s) (6000 8%) 5.3559
9m 34s (- 93m 2s) (7000 9%) 5.3391
10m 59s (- 91m 59s) (8000 10%) 5.3865
12m 17s (- 90m 11s) (9000 12%) 5.1766
13m 39s (- 88m 49s) (10000 13%) 5.3228
15m 4s (- 87m 39s) (11000 14%) 5.3723
16m 28s (- 86m 30s) (12000 16%) 5.4256
17m 52s (- 85m 14s) (13000 17%) 5.4295
19m 15s (- 83m 54s) (14000 18%) 5.4926
20m 38s (- 82m 34s) (15000 20%) 5.4873
21m 59s (- 81m 5s) (16000 21%) 5.3890
23m 24s (- 79m 52s) (17000 22%) 5.5100
24m 51s (- 78m 42s) (18000 24%) 5.5544
26m 16s (- 77m 25s) (19000 25%) 5.5492
27m 42s (- 76m 13s) (20000 26%) 5.5808
29m 6s (- 74m 51s) (21000 28%) 5.4492
30m 30s (- 73m 30s) (22000 29%) 5.4833
31m 53s (- 72m 6s) (23000 30%) 5.4163
33m 19s (- 70m 48s) (24000 32%) 5.4506
34m 42s (- 69m 25s) (25000 33%) 5.3769
36m 9s (- 68m 9s) (26000 34%) 5.4788
37m 37s (- 66m 5

###Fold 4

In [18]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
MAX_LENGTH = 171
# Multiplicative attention https://blog.floydhub.com/attention-mechanism/
class AttnDecoderRNN4(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN4, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        #################
        self.fc = nn.Linear(hidden_size, hidden_size, bias=False) #  Multiplicative
        ##################
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        
        x=self.fc(hidden)
        #print(x.view(1,-1,1).shape,encoder_outputs.unsqueeze(0).shape )
        attn_weights = F.softmax(
            #################################
             encoder_outputs.unsqueeze(0).bmm(x.view(1,-1,1)).view(1,-1), dim=1)
            #################################
       # print(attn_weights.view(1,-1,1).shape,encoder_outputs.unsqueeze(0).shape)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
        
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ') if word in lang.word2index]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang4, pair[0])
    target_tensor = tensorFromSentence(output_lang4, pair[1])
    return (input_tensor, target_tensor)

teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_pairs4))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [19]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang4, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang4.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self, alpha=1.0):
        reward = 0
        # Add here a function for shaping a reward

        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward
    
    def __lt__(self, other):
        return self.eval() < other.eval()
    
    
from queue import PriorityQueue

def evaluate_beam_search(encoder, decoder, sentence, max_length=MAX_LENGTH, beam_size=2):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang4, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        # Number of sentence to generate
        endnodes = []
        number_required = 1
        
        # starting node -  hidden vector, previous node, word id, logp, length
        node = BeamSearchNode(decoder_hidden, None, decoder_input, 0, 1)
        nodes = PriorityQueue()

        # start the queue
        nodes.put((-node.eval(), node))
        qsize = 1
        
        # start beam search
        while True:
            # give up when decoding takes too long
            if qsize > 2000: break

            # fetch the best node
            score, n = nodes.get()
            decoder_input = n.wordid
            decoder_hidden = n.h

            if n.wordid.item() == EOS_token and n.prevNode != None:
                endnodes.append((score, n))
                # if we reached maximum # of sentences required
                if len(endnodes) >= number_required:
                    break
                else:
                    continue
            #elif n.leng > max_length:
            #    continue

            # decode for one step using decoder
            decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_outputs)

            # PUT HERE REAL BEAM SEARCH OF TOP
            log_prob, indexes = torch.topk(decoder_output, beam_size)
            nextnodes = []

            for new_k in range(beam_size):
                decoded_t = indexes[0][new_k].view(1, -1)
                log_p = log_prob[0][new_k].item()

                node = BeamSearchNode(decoder_hidden, n, decoded_t, n.logp + log_p, n.leng + 1)
                score = -node.eval()
                nextnodes.append((score, node))

            # put them into queue
            for i in range(len(nextnodes)):
                score, nn = nextnodes[i]
                nodes.put((score, nn))
                
            # increase qsize
            qsize += len(nextnodes) - 1
            
        # choose nbest paths, back trace them
        if len(endnodes) == 0:
            endnodes = [nodes.get() for _ in range(number_required)]

        _, n = endnodes[0]
        utterance = []
        utterance.append(output_lang4.index2word[n.wordid.item()])
        
        # back trace
        while n.prevNode != None:
            n = n.prevNode
            utterance.append(output_lang4.index2word[n.wordid.item()])

        utterance = utterance[::-1]
            
    return utterance, None

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(test_pairs4)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

from nltk.translate.bleu_score import corpus_bleu

def evaluateBleu(encoder, decoder):
    references, candidates = [], []
    i= 0
    for sent_eng, sents_fre in test_pairs4:
        i=i+1
        sents_fre = [sent_fre.split(' ') for sent_fre in [sents_fre]]
        output_words, _ = evaluate(encoder, decoder, sent_eng)
        references.append(sents_fre)
        candidates.append(output_words)
        if i%1000==0:
          print(i)
    #return bleu 1, bleu 2, bleu 3
    score1 = corpus_bleu(references, candidates, weights=(1.0, 0, 0))
    score2 = corpus_bleu(references, candidates, weights=(0.5, 0.5, 0))
    score3 = corpus_bleu(references, candidates, weights=(0.33, 0.33, 0.33))
    #print (references,  candidates)
    scores = [score1, score2, score3]
    return scores

from nltk.translate.bleu_score import corpus_bleu

def evaluateBleu_beam_search(encoder, decoder, beam_size):
    references, candidates = [], []
    for sent_eng, sents_fre in test_pairs4:
        sents_fre = [sent_fre.split(' ') for sent_fre in [sents_fre]]
        output_words, _ = evaluate_beam_search(encoder, decoder, sent_eng, beam_size=beam_size)
        references.append(sents_fre)
        candidates.append(output_words)
    #return bleu 1, bleu 2, bleu 3
    score1 = corpus_bleu(references, candidates, weights=(1.0, 0, 0))
    score2 = corpus_bleu(references, candidates, weights=(0.5, 0.5, 0))
    score3 = corpus_bleu(references, candidates, weights=(0.33, 0.33, 0.33))
    #print (references,  candidates)
    scores = [score1, score2, score3]
    return scores

In [20]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang4.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN4(hidden_size, output_lang4.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=1000) #reduce number of epoch
print("Fold 4 Bleu-1, Bleu-2, Bleu-3 scores are ",evaluateBleu(encoder1, attn_decoder1))

1m 27s (- 107m 46s) (1000 1%) 6.1335
2m 48s (- 102m 44s) (2000 2%) 5.5939
4m 8s (- 99m 21s) (3000 4%) 5.3221
5m 27s (- 97m 1s) (4000 5%) 5.3612
6m 47s (- 95m 3s) (5000 6%) 5.3687
8m 9s (- 93m 44s) (6000 8%) 5.3038
9m 31s (- 92m 33s) (7000 9%) 5.3091
10m 54s (- 91m 19s) (8000 10%) 5.3746
12m 16s (- 90m 1s) (9000 12%) 5.4580
13m 36s (- 88m 29s) (10000 13%) 5.3592
14m 57s (- 87m 2s) (11000 14%) 5.2833
16m 20s (- 85m 46s) (12000 16%) 5.3737
17m 41s (- 84m 21s) (13000 17%) 5.3713
19m 3s (- 83m 2s) (14000 18%) 5.3134
20m 24s (- 81m 37s) (15000 20%) 5.3801
21m 49s (- 80m 28s) (16000 21%) 5.3295
23m 13s (- 79m 14s) (17000 22%) 5.4585
24m 38s (- 78m 2s) (18000 24%) 5.5768
26m 5s (- 76m 53s) (19000 25%) 5.4791
27m 28s (- 75m 32s) (20000 26%) 5.4239
28m 52s (- 74m 16s) (21000 28%) 5.5030
30m 16s (- 72m 57s) (22000 29%) 5.4351
31m 40s (- 71m 36s) (23000 30%) 5.4443
33m 4s (- 70m 16s) (24000 32%) 5.4719
34m 30s (- 69m 0s) (25000 33%) 5.4917
35m 55s (- 67m 42s) (26000 34%) 5.4868
37m 20s (- 66m 22s)

###Fold 5

In [21]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
MAX_LENGTH = 171
# Multiplicative attention https://blog.floydhub.com/attention-mechanism/
class AttnDecoderRNN4(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN4, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        #################
        self.fc = nn.Linear(hidden_size, hidden_size, bias=False) #  Multiplicative
        ##################
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        
        x=self.fc(hidden)
        #print(x.view(1,-1,1).shape,encoder_outputs.unsqueeze(0).shape )
        attn_weights = F.softmax(
            #################################
             encoder_outputs.unsqueeze(0).bmm(x.view(1,-1,1)).view(1,-1), dim=1)
            #################################
       # print(attn_weights.view(1,-1,1).shape,encoder_outputs.unsqueeze(0).shape)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
        
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
        
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ') if word in lang.word2index]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang5, pair[0])
    target_tensor = tensorFromSentence(output_lang5, pair[1])
    return (input_tensor, target_tensor)

teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_pairs5))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [22]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang5, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang5.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self, alpha=1.0):
        reward = 0
        # Add here a function for shaping a reward

        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward
    
    def __lt__(self, other):
        return self.eval() < other.eval()
    
    
from queue import PriorityQueue

def evaluate_beam_search(encoder, decoder, sentence, max_length=MAX_LENGTH, beam_size=2):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang5, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        # Number of sentence to generate
        endnodes = []
        number_required = 1
        
        # starting node -  hidden vector, previous node, word id, logp, length
        node = BeamSearchNode(decoder_hidden, None, decoder_input, 0, 1)
        nodes = PriorityQueue()

        # start the queue
        nodes.put((-node.eval(), node))
        qsize = 1
        
        # start beam search
        while True:
            # give up when decoding takes too long
            if qsize > 2000: break

            # fetch the best node
            score, n = nodes.get()
            decoder_input = n.wordid
            decoder_hidden = n.h

            if n.wordid.item() == EOS_token and n.prevNode != None:
                endnodes.append((score, n))
                # if we reached maximum # of sentences required
                if len(endnodes) >= number_required:
                    break
                else:
                    continue
            #elif n.leng > max_length:
            #    continue

            # decode for one step using decoder
            decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_outputs)

            # PUT HERE REAL BEAM SEARCH OF TOP
            log_prob, indexes = torch.topk(decoder_output, beam_size)
            nextnodes = []

            for new_k in range(beam_size):
                decoded_t = indexes[0][new_k].view(1, -1)
                log_p = log_prob[0][new_k].item()

                node = BeamSearchNode(decoder_hidden, n, decoded_t, n.logp + log_p, n.leng + 1)
                score = -node.eval()
                nextnodes.append((score, node))

            # put them into queue
            for i in range(len(nextnodes)):
                score, nn = nextnodes[i]
                nodes.put((score, nn))
                
            # increase qsize
            qsize += len(nextnodes) - 1
            
        # choose nbest paths, back trace them
        if len(endnodes) == 0:
            endnodes = [nodes.get() for _ in range(number_required)]

        _, n = endnodes[0]
        utterance = []
        utterance.append(output_lang5.index2word[n.wordid.item()])
        
        # back trace
        while n.prevNode != None:
            n = n.prevNode
            utterance.append(output_lang5.index2word[n.wordid.item()])

        utterance = utterance[::-1]
            
    return utterance, None

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(test_pairs5)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

from nltk.translate.bleu_score import corpus_bleu

def evaluateBleu(encoder, decoder):
    references, candidates = [], []
    i= 0
    for sent_eng, sents_fre in test_pairs5:
        i=i+1
        sents_fre = [sent_fre.split(' ') for sent_fre in [sents_fre]]
        output_words, _ = evaluate(encoder, decoder, sent_eng)
        references.append(sents_fre)
        candidates.append(output_words)
        if i%1000==0:
          print(i)
    #return bleu 1, bleu 2, bleu 3
    score1 = corpus_bleu(references, candidates, weights=(1.0, 0, 0))
    score2 = corpus_bleu(references, candidates, weights=(0.5, 0.5, 0))
    score3 = corpus_bleu(references, candidates, weights=(0.33, 0.33, 0.33))
    #print (references,  candidates)
    scores = [score1, score2, score3]
    return scores

from nltk.translate.bleu_score import corpus_bleu

def evaluateBleu_beam_search(encoder, decoder, beam_size):
    references, candidates = [], []
    for sent_eng, sents_fre in test_pairs5:
        sents_fre = [sent_fre.split(' ') for sent_fre in [sents_fre]]
        output_words, _ = evaluate_beam_search(encoder, decoder, sent_eng, beam_size=beam_size)
        references.append(sents_fre)
        candidates.append(output_words)
    #return bleu 1, bleu 2, bleu 3
    score1 = corpus_bleu(references, candidates, weights=(1.0, 0, 0))
    score2 = corpus_bleu(references, candidates, weights=(0.5, 0.5, 0))
    score3 = corpus_bleu(references, candidates, weights=(0.33, 0.33, 0.33))
    #print (references,  candidates)
    scores = [score1, score2, score3]
    return scores

In [23]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang5.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN4(hidden_size, output_lang5.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=1000) #reduce number of epoch
print("Fold 5 Bleu-1, Bleu-2, Bleu-3 scores are ",evaluateBleu(encoder1, attn_decoder1))

1m 24s (- 104m 34s) (1000 1%) 5.9733
2m 47s (- 101m 44s) (2000 2%) 5.8299
4m 8s (- 99m 16s) (3000 4%) 5.5017
5m 28s (- 97m 11s) (4000 5%) 5.4256
6m 47s (- 95m 2s) (5000 6%) 5.3052
8m 8s (- 93m 35s) (6000 8%) 5.4052
9m 31s (- 92m 31s) (7000 9%) 5.4877
10m 53s (- 91m 12s) (8000 10%) 5.4281
12m 15s (- 89m 55s) (9000 12%) 5.3511
13m 35s (- 88m 23s) (10000 13%) 5.3761
15m 1s (- 87m 27s) (11000 14%) 5.4865
16m 26s (- 86m 21s) (12000 16%) 5.5096
17m 51s (- 85m 10s) (13000 17%) 5.4265
19m 16s (- 83m 59s) (14000 18%) 5.4952
20m 41s (- 82m 45s) (15000 20%) 5.5617
22m 5s (- 81m 28s) (16000 21%) 5.5874
23m 31s (- 80m 15s) (17000 22%) 5.5908
24m 58s (- 79m 5s) (18000 24%) 5.5780
26m 24s (- 77m 49s) (19000 25%) 5.5485
27m 51s (- 76m 36s) (20000 26%) 5.5138
29m 17s (- 75m 20s) (21000 28%) 5.5494
30m 43s (- 74m 1s) (22000 29%) 5.4680
32m 8s (- 72m 40s) (23000 30%) 5.4619
33m 33s (- 71m 18s) (24000 32%) 5.5376
35m 1s (- 70m 3s) (25000 33%) 5.4876
36m 28s (- 68m 44s) (26000 34%) 5.5035
37m 54s (- 67m 23

In [24]:
evaluateRandomly(encoder1, attn_decoder1)

> Не обязательно .

= not necessarily .
< nor is not . <EOS>

> Еще более интригующим является то, что один ген может ввести в действие этот механизм выживания у целого ряда живых существ . 

= but in times of scarcity the survival program kicks in to slow the aging process . even more intriguingly a single gene can promote this survival mechanism across a wide swath of nature s creatures .
< indeed many people that the world s many of the or <EOS>

> Многим людям всегда кажется, что “на этот раз все по-другому” .

= for many people it always seems that this time is different . 
< the is that is that is to . . <EOS>

> Тогда как полностью скопировать региональный опыт невозможно, североевропейским странам удалось успешно совместить социальное обеспечение с высоким уровнем доходов, стабильным экономическим ростом и макроэкономической стабильностью .

= while no regional experience is directly transferable the nordic countries have successfully combined social welfare with high income le